In [1]:
import statsmodels.api 
import pandas as pd
import numpy as np
import torch
from   torch.utils.data import TensorDataset, DataLoader
from   torch import nn
from   torch.nn import functional as f

In [38]:
from sklearn.datasets import load_iris

In [39]:
X, y = load_iris(return_X_y=True)

In [40]:
X.shape

(150, 4)

In [60]:
model = nn.Sequential(
    nn.BatchNorm1d(num_features=4),
    nn.Linear(4, 9),
    nn.ELU(),
    nn.Dropout(p=0.1),
    nn.Linear(9, 6),
    nn.ELU(),
    nn.Linear(6, 3),
    nn.Softmax()
)

In [61]:
X_ = torch.tensor(X, dtype=torch.float32)
y_ = torch.tensor(np.eye(3)[y], dtype=torch.float32)


In [62]:
train = TensorDataset(X_, y_)
train = DataLoader(
    dataset = train, batch_size=32, shuffle=True
)

In [68]:
loss_ = nn.CrossEntropyLoss()
optim = torch.optim.RMSprop(
    model.parameters()
)

for _ in range(10):
    for xbatch, ybatch in train:

        pred = model(xbatch)
        loss = loss_(ybatch, pred)
        

        optim.zero_grad()
        loss.backward()
        optim.step()
    print(loss.item())

0.6399635076522827
0.6003660559654236
0.602482795715332
0.6824911832809448
0.6156812310218811
0.6242931485176086
0.6831119060516357
0.6988590359687805
0.5802111625671387
0.7603246569633484


In [69]:
from sklearn.metrics import *
with torch.no_grad():
    pred = model(X_)

In [70]:
torch.argmax(pred, dim=1)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1,
        1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2])

In [71]:
print(classification_report(y, torch.argmax(pred, dim=1)))

              precision    recall  f1-score   support

           0       1.00      0.98      0.99        50
           1       0.94      0.96      0.95        50
           2       0.96      0.96      0.96        50

    accuracy                           0.97       150
   macro avg       0.97      0.97      0.97       150
weighted avg       0.97      0.97      0.97       150

